In [1]:
import os
import sys

sys.path.append('..')

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [4]:
attacks = ['fgsm', 'apgd', 'apgd2', 'cw2', 'deepfool', 'line']

list_df = []
for i in range(5):
    for a in attacks:
        p_mnist_dnn_apgd2_baard = os.path.join(
            '..','results', 'result_{}'.format(i), 'mnist_dnn', 'results', 'mnist_dnn_{}_baard.csv'.format(a))
        df = pd.read_csv(p_mnist_dnn_apgd2_baard)
        df = df.drop('Unnamed: 0', axis=1)
        list_df.append(df)
df = pd.concat(list_df, axis=0, ignore_index=True)
df_copy = df.copy()
df_mnist_baard = df.groupby(['data', 'model', 'attack', 'adv_param'], as_index=False).agg({
    'acc_no_def': ['mean', 'std'],
    'acc_on_adv_1': ['mean', 'std'],
    'fpr_1': ['mean', 'std'],
    'acc_on_adv_2': ['mean', 'std'],
    'fpr_2': ['mean', 'std'],
    'acc_on_adv_3': ['mean', 'std'],
    'fpr_3': ['mean', 'std'],
})
df_mnist_baard

data model    attack adv_param acc_no_def           acc_on_adv_1  \
                                          mean       std         mean   
0   mnist   dnn      apgd     0.063     0.8618  0.004817       1.0000   
1   mnist   dnn      apgd     0.100     0.5222  0.022477       1.0000   
2   mnist   dnn      apgd     0.300     0.0000  0.000000       1.0000   
3   mnist   dnn      apgd     0.600     0.0000  0.000000       1.0000   
4   mnist   dnn      apgd     1.000     0.0000  0.000000       1.0000   
5   mnist   dnn      apgd     1.500     0.0000  0.000000       1.0000   
6   mnist   dnn      apgd     2.000     0.0000  0.000000       1.0000   
7   mnist   dnn     apgd2     1.000     0.7680  0.007874       1.0000   
8   mnist   dnn     apgd2     1.500     0.3988  0.016604       1.0000   
9   mnist   dnn     apgd2     2.000     0.1606  0.007335       1.0000   
10  mnist   dnn     apgd2     3.000     0.0852  0.009859       1.0000   
11  mnist   dnn     apgd2     4.000     0.0754  0.005505       1.0000   
12  mnist   dnn     apgd2     5.000     0.0606  0.012779       1.0000   
13  mnist   dnn       cw2     0.000     0.0000  0.000000       1.0000   
14  mnist   dnn       cw2     5.000     0.0000  0.000000       1.0000   
15  mnist   dnn       cw2    10.000     0.0000  0.000000       1.0000   
16  mnist   dnn  deepfool     0.000     0.0324  0.007162       1.0000   
17  mnist   dnn      fgsm     0.063     0.8958  0.005404       1.0000   
18  mnist   dnn      fgsm     0.100     0.6650  0.021131       1.0000   
19  mnist   dnn      fgsm     0.300     0.0698  0.005630       1.0000   
20  mnist   dnn      fgsm     0.600     0.0088  0.006907       1.0000   
21  mnist   dnn      fgsm     1.000     0.0070  0.003240       1.0000   
22  mnist   dnn      fgsm     1.500     0.0070  0.003240       1.0000   
23  mnist   dnn      fgsm     2.000     0.0070  0.003240       1.0000   
24  mnist   dnn      line     1.000     0.9736  0.006387       0.9878   

             fpr_1           acc_on_adv_2             fpr_2            \
        std   mean       std         mean       std    mean       std   
0   0.00000  0.006  0.002345       1.0000  0.000000  0.0176  0.002881   
1   0.00000  0.006  0.002345       1.0000  0.000000  0.0176  0.002881   
2   0.00000  0.006  0.002345       1.0000  0.000000  0.0176  0.002881   
3   0.00000  0.006  0.002345       1.0000  0.000000  0.0176  0.002881   
4   0.00000  0.006  0.002345       1.0000  0.000000  0.0176  0.002881   
5   0.00000  0.006  0.002345       1.0000  0.000000  0.0176  0.002881   
6   0.00000  0.006  0.002345       1.0000  0.000000  0.0176  0.002881   
7   0.00000  0.006  0.002345       1.0000  0.000000  0.0176  0.002881   
8   0.00000  0.006  0.002345       1.0000  0.000000  0.0176  0.002881   
9   0.00000  0.006  0.002345       1.0000  0.000000  0.0176  0.002881   
10  0.00000  0.006  0.002345       1.0000  0.000000  0.0176  0.002881   
11  0.00000  0.006  0.002345       1.0000  0.000000  0.0176  0.002881   
12  0.00000  0.006  0.002345       1.0000  0.000000  0.0176  0.002881   
13  0.00000  0.006  0.002345       1.0000  0.000000  0.0176  0.002881   
14  0.00000  0.006  0.002345       1.0000  0.000000  0.0176  0.002881   
15  0.00000  0.006  0.002345       1.0000  0.000000  0.0176  0.002881   
16  0.00000  0.006  0.002345       1.0000  0.000000  0.0176  0.002881   
17  0.00000  0.006  0.002345       1.0000  0.000000  0.0176  0.002881   
18  0.00000  0.006  0.002345       1.0000  0.000000  0.0176  0.002881   
19  0.00000  0.006  0.002345       1.0000  0.000000  0.0176  0.002881   
20  0.00000  0.006  0.002345       1.0000  0.000000  0.0176  0.002881   
21  0.00000  0.006  0.002345       1.0000  0.000000  0.0176  0.002881   
22  0.00000  0.006  0.002345       1.0000  0.000000  0.0176  0.002881   
23  0.00000  0.006  0.002345       1.0000  0.000000  0.0176  0.002881   
24  0.00638  0.006  0.002345       0.9898  0.005119  0.0176  0.002881   

   acc_on_adv_3             fpr_3            
           mean

In [5]:
baard_s3 = df_copy.rename(columns={"acc_on_adv_3": "acc_on_adv", "fpr_3": "fpr"})
baard_s3['defence'] = 'baard'
df_mnist_baard = baard_s3.groupby(['data', 'model', 'attack', 'adv_param', 'defence'], as_index=False).agg({
    'acc_no_def': ['mean', 'std'],
    'acc_on_adv': ['mean', 'std'],
    'fpr': ['mean', 'std']
})
df_mnist_baard

data model    attack adv_param defence acc_no_def           acc_on_adv  \
                                                  mean       std       mean   
0   mnist   dnn      apgd     0.063   baard     0.8618  0.004817     1.0000   
1   mnist   dnn      apgd     0.100   baard     0.5222  0.022477     1.0000   
2   mnist   dnn      apgd     0.300   baard     0.0000  0.000000     1.0000   
3   mnist   dnn      apgd     0.600   baard     0.0000  0.000000     1.0000   
4   mnist   dnn      apgd     1.000   baard     0.0000  0.000000     1.0000   
5   mnist   dnn      apgd     1.500   baard     0.0000  0.000000     1.0000   
6   mnist   dnn      apgd     2.000   baard     0.0000  0.000000     1.0000   
7   mnist   dnn     apgd2     1.000   baard     0.7680  0.007874     1.0000   
8   mnist   dnn     apgd2     1.500   baard     0.3988  0.016604     1.0000   
9   mnist   dnn     apgd2     2.000   baard     0.1606  0.007335     1.0000   
10  mnist   dnn     apgd2     3.000   baard     0.0852  0.009859     1.0000   
11  mnist   dnn     apgd2     4.000   baard     0.0754  0.005505     1.0000   
12  mnist   dnn     apgd2     5.000   baard     0.0606  0.012779     1.0000   
13  mnist   dnn       cw2     0.000   baard     0.0000  0.000000     1.0000   
14  mnist   dnn       cw2     5.000   baard     0.0000  0.000000     1.0000   
15  mnist   dnn       cw2    10.000   baard     0.0000  0.000000     1.0000   
16  mnist   dnn  deepfool     0.000   baard     0.0324  0.007162     1.0000   
17  mnist   dnn      fgsm     0.063   baard     0.8958  0.005404     1.0000   
18  mnist   dnn      fgsm     0.100   baard     0.6650  0.021131     1.0000   
19  mnist   dnn      fgsm     0.300   baard     0.0698  0.005630     1.0000   
20  mnist   dnn      fgsm     0.600   baard     0.0088  0.006907     1.0000   
21  mnist   dnn      fgsm     1.000   baard     0.0070  0.003240     1.0000   
22  mnist   dnn      fgsm     1.500   baard     0.0070  0.003240     1.0000   
23  mnist   dnn      fgsm     2.000   baard     0.0070  0.003240     1.0000   
24  mnist   dnn      line     1.000   baard     0.9736  0.006387     0.9944   

                 fpr            
         std    mean       std  
0   0.000000  0.0272  0.003271  
1   0.000000  0.0272  0.003271  
2   0.000000  0.0272  0.003271  
3   0.000000  0.0272  0.003271  
4   0.000000  0.0272  0.003271  
5   0.000000  0.0272  0.003271  
6   0.000000  0.0272  0.003271  
7   0.000000  0.0272  0.003271  
8   0.000000  0.0272  0.003271  
9   0.000000  0.0272  0.003271  
10  0.000000  0.0272  0.003271  
11  0.000000  0.0272  0.003271  
12  0.000000  0.0272  0.003271  
13  0.000000  0.0272  0.003271  
14  0.000000  0.0272  0.003271  
15  0.000000  0.0272  0.003271  
16  0.000000  0.0272  0.003271  
17  0.000000  0.0272  0.003271  
18  0.000000  0.0272  0.003271  
19  0.000000  0.0272  0.003271  
20  0.000000  0.0272  0.003271  
21  0.000000  0.0272  0.003271  
22  0.000000  0.0272  0.003271  
23  0.000000  0.0272  0.003271  
24  0.002793  0.0272  0.003271

In [6]:
attacks = ['fgsm', 'apgd', 'apgd2', 'cw2', 'deepfool', 'line']
defences = ['fs', 'lid', 'magnet', 'rc']

list_df = []
for i in range(5):
    for a in attacks:
        for d in defences:
            p_mnist_dnn_apgd2_baard = os.path.join(
                '..', 'results', 'result_{}'.format(i), 'mnist_dnn', 'results', 'mnist_dnn_{}_{}.csv'.format(a, d))
            df = pd.read_csv(p_mnist_dnn_apgd2_baard)
            df = df.drop('Unnamed: 0', axis=1)
            df['defence'] = d
            list_df.append(df)
df = pd.concat(list_df, axis=0, ignore_index=True)
df_mnist_others = df.groupby(['data', 'model', 'attack', 'adv_param', 'defence'], as_index=False).agg({
    'acc_no_def': ['mean', 'std'],
    'acc_on_adv': ['mean', 'std'],
    'fpr': ['mean', 'std']
})

In [7]:
df = pd.concat([df_mnist_baard, df_mnist_others], axis=0, ignore_index=True)
df.columns = ["_".join(x) if x[1] != '' else x[0] for x in df.columns.ravel()]
df.head()

,data,model,attack,adv_param,defence,acc_no_def_mean,acc_no_def_std,acc_on_adv_mean,acc_on_adv_std,fpr_mean,fpr_std
0,mnist,dnn,apgd,0.063,baard,0.8618,0.004817,1.0,0.0,0.0272,0.003271
1,mnist,dnn,apgd,0.100,baard,0.5222,0.022477,1.0,0.0,0.0272,0.003271
2,mnist,dnn,apgd,0.300,baard,0.0000,0.000000,1.0,0.0,0.0272,0.003271
3,mnist,dnn,apgd,0.600,baard,0.0000,0.000000,1.0,0.0,0.0272,0.003271
4,mnist,dnn,apgd,1.000,baard,0.0000,0.000000,1.0,0.0,0.0272,0.003271


In [8]:
df = df.sort_values(by=['data', 'model', 'attack', 'adv_param', 'acc_on_adv_mean', 'fpr_mean'], 
               ascending=[True, True, True, True, False, True])
df

,data,model,attack,adv_param,defence,acc_no_def_mean,acc_no_def_std,acc_on_adv_mean,acc_on_adv_std,fpr_mean,fpr_std
27,mnist,dnn,apgd,0.063,magnet,0.8618,0.004817,1.0000,0.000000,0.0028,0.002168
0,mnist,dnn,apgd,0.063,baard,0.8618,0.004817,1.0000,0.000000,0.0272,0.003271
25,mnist,dnn,apgd,0.063,fs,0.8618,0.004817,0.9138,0.010986,0.0528,0.012558
28,mnist,dnn,apgd,0.063,rc,0.8618,0.004817,0.8952,0.009985,0.0008,0.000837
26,mnist,dnn,apgd,0.063,lid,0.8618,0.004817,0.8618,0.004817,0.0000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
24,mnist,dnn,line,1.000,baard,0.9736,0.006387,0.9944,0.002793,0.0272,0.003271
123,mnist,dnn,line,1.000,magnet,0.9736,0.006387,0.9892,0.006535,0.0028,0.002168
121,mnist,dnn,line,1.000,fs,0.9736,0.006387,0.9824,0.005413,0.0528,0.012558
122,mnist,dnn,line,1.000,lid,0.9736,0.006387,0.9736,0.006387,0.0000,0.000000


In [9]:
df.to_csv('mnist_results.csv', index=False)